# IBA Probe for Hot/Cold Interface

In this tutorial we will demonstrate the "Hot/Cold Interface" probe that is used to messure specific traffic load on your interfaces and detect when traffic load increases (hot) or decreses (cold) unexpectedly.

The first thing we need to do to is setup our AOS server and identify the blueprint we want to use.

In [ ]:
aos_server = '172.20.62.3'
blueprint_name = "rack-based-blueprint-3b245327"

Next let's create a AOS python client that we will use to create our probe

In [ ]:
import json
from aospy.client import Client

aos = Client(aos_server)


Now let's use that client to get the blueprint, so we can have the blueprint_id value we will need when we post the probe.

In [ ]:
bp = aos.blueprints[blueprint_name]

In [ ]:
print bp.id

Now let's import the hot/cold interface probe from the library

In [ ]:
from aospy.ibaprobelib.hot_cold_interface import hotcold_ifcounter_probe

Next let's create the hot/cold interface probe payload information that we will need.  The probe takes a few parmateres which we can see if we introspect the function:

In [ ]:
?hotcold_ifcounter_probe

The following paramaters are required:

`label` - Descriptive name for the probe  
`if_counter` - Type of interface counter:  
    - tx_multicast_packets, rx_bytes, rx_error_packets, tx_error_packets,   
    - tx_unicast_packets, rx_discard_packets, rx_broadcast_packets,   
    - tx_broadcast_packets, rx_multicast_packets, tx_bytes, tx_discard_packets,   
    - rx_unicast_packets 
      
`min` - minimum level of counter  
`max` - maximum level of counter  
`max_hot_interface_percentage` - maximum percentage of hot interfaces on a device  
`max_cold_interface_percentage` - maximum percentage of cold interfaces on a device  
`average_period` - period over which to average input counter samples  
`duration` - number of seconds of recent-history over which interface counter hot/cold  
status will be considered  
`threshold_duration` - sum total of number of seconds of recent-history for which  
interface must be hot/cold for anomaly to be raised  
`anomaly_history_count` - number of anomaly flaps that will be recorded for inspection  


So for the case of reuse, we will define these variables here:

In [ ]:
probe_label = 'DC1-hot-cold-int'
if_counter = 'tx_unicast_packets'
min = 12500
max = 12500000
max_hot_interface_percentage = 5
max_cold_interface_percentage = 5
average_period = 30
duration = 60
threshold_duration = 120
anomaly_history_count = 500

Now let's create the body data from these variables:

In [ ]:
probe_body = hotcold_ifcounter_probe(label=probe_label, if_counter=if_counter, min=min, max=max, max_hot_interface_percentage=max_hot_interface_percentage, max_cold_interface_percentage=max_cold_interface_percentage, average_period=average_period, duration=duration, threshold_duration=threshold_duration, anomaly_history_count=anomaly_history_count)

If you'd like to dump the contents of the probe_body, you can use the following:

In [ ]:
# print json.dumps(probe_body, indent=2)

Finally, post the probe body to AOS to create the probe.

In [ ]:
probes_url = aos.url + "/api/blueprints/{blueprint_id}/probes".format(blueprint_id=bp.id)

In [ ]:
probes_url

In [ ]:
resp = aos.api.post(probes_url, json=probe_body)

In [ ]:
resp.ok

Now we have our IBA Probe ID:

In [ ]:
probe_id = resp.json()['id']

print probe_id

And now we can see the IBA active probe in the UI:

<img src="media/hotcold_interface1.png">

And then select the probe on the UI to get the details:

<img src="media/hotcold_interface2.png">

# Delete the IBA ECMP Traffic Imbalance Probe

If you want to remove the IBA probe, you can use the API to do so, like the following; just uncomment this out.

In [ ]:
# resp = aos.api.delete(probes_url + "/%s" % probe_id)
# resp.ok